1. 单系列柱状图
2. 多系列柱状图
3. 堆叠图
4. 直方图

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [18]:
# 1、单系列柱状图
# vbar

p = figure(plot_width=400, plot_height=400)
p.vbar(x=[1, 2, 3],
       width=0.5,
       bottom=0,
       top=[1.2, 2.5, 3.7],
       line_width=1,
       line_alpha=0.8,
       line_color='black',
       line_dash=[10, 6],
       fill_color='red',
       fill_alpha=0.6)
# x：横轴坐标，width：柱子宽度，bottom：底高度，top：顶高度
#color = ['red','blue','green'], alpha = 0.8   # 整体颜色设置，也可单独设置 → color="firebrick"

show(p)

In [19]:
# 1、单系列柱状图
# hbar

p = figure(plot_width=400, plot_height=400)
p.hbar(y=[1, 2, 3],
       height=0.5,
       left=0,
       right=[1.2, 2.5, 3.7],
       color=['red', 'blue', 'green'])

show(p)

In [20]:
# 1、单系列柱状图 - 分类设置标签
# ColumnDataSource

from bokeh.palettes import Spectral6
from bokeh.transform import factor_cmap

fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
counts = [5, 3, 4, 2, 4, 6]
source = ColumnDataSource(data=dict(fruits=fruits, counts=counts))
colors = ["salmon", "olive", "darkred", "goldenrod", "skyblue", "orange"]
# 创建一个包含标签的data，对象类型为ColumnDataSource

In [21]:
source.to_df()

,fruits,counts
0,Apples,5
1,Pears,3
2,Nectarines,4
3,Plums,2
4,Grapes,4
5,Strawberries,6


In [22]:
p = figure(x_range=fruits,
           y_range=(0, 9),
           plot_height=350,
           title='Fruit Counts',
           tools='')
p.vbar(x='fruits',
       top='counts',
       source=source,
       width=0.9,
       alpha=0.8,
       color=factor_cmap('fruits', palette=Spectral6, factors=fruits),
       legend='fruits')
# 绘制柱状图，横轴直接显示标签
# factor_cmap(field_name, palette, factors, start=0, end=None, nan_color='gray')：颜色转换模块，生成一个颜色转换对象
# field_name：分类名称
# palette：调色盘
# factors：用于在调色盘中分颜色的参数
# 参考文档：http://bokeh.pydata.org/en/latest/docs/reference/transform.html

p.xgrid.grid_line_color = None
p.legend.orientation = 'horizontal'
p.legend.location = 'top_center'

show(p)

In [23]:
# 2、多系列柱状图
# vbar

# 导入dodge、value模块
from bokeh.transform import dodge
from bokeh.core.properties import value

# 创建数据
df = pd.DataFrame(
    {
        '2015': [2, 1, 4, 3, 2, 4],
        '2016': [5, 3, 3, 2, 4, 6],
        '2017': [3, 2, 4, 4, 5, 3]
    },
    index=['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries'])

fruit = df.index.tolist()
years = df.columns.tolist()
data = {'index': fruits}
for year in years:
    data[year] = df[year].tolist()

print(data)

{'index': ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries'], '2015': [2, 1, 4, 3, 2, 4], '2016': [5, 3, 3, 2, 4, 6], '2017': [3, 2, 4, 4, 5, 3]}


In [24]:
source = ColumnDataSource(data=data)

In [25]:
p = figure(x_range=fruits,
           y_range=(0, 10),
           plot_height=350,
           title='Fruit count by year',
           tools='')
p.vbar(x=dodge('index', -0.25, range=p.x_range),
       top='2015',
       width=0.2,
       source=source,
       color='#c9d9d3',
       legend=value('2015'))
p.vbar(x=dodge('index', 0, range=p.x_range),
       top='2016',
       width=0.2,
       source=source,
       color='#718dbf',
       legend=value('2016'))
p.vbar(x=dodge('index', 0.25, range=p.x_range),
       top='2017',
       width=0.2,
       source=source,
       color='#e84d60',
       legend=value('2017'))
# 绘制多系列柱状图
# dodge(field_name, value, range=None) → 转换成一个可分组的对象，value为元素的位置（配合width设置）
# value(val, transform=None) → 按照年份分为dict

p.xgrid.grid_line_color = None
p.legend.location = 'top_left'
p.legend.orientation = 'horizontal'

show(p)

In [26]:
# 3、堆叠图

from bokeh.core.properties import value
# 导入value模块

fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
years = ["2015", "2016", "2017"]
colors = ["#c9d9d3", "#718dbf", "#e84d60"]
data = {
    'fruits': fruits,
    '2015': [2, 1, 4, 3, 2, 4],
    '2016': [5, 3, 4, 2, 4, 6],
    '2017': [3, 2, 4, 4, 5, 3]
}
source = ColumnDataSource(data=data)
# 创建数据

In [27]:
p = figure(x_range=fruits,
           y_range=(0, 15),
           plot_height=400,
           title='Fruit count by year',
           tools='')
p.vbar_stack(
    years,  # 设置堆叠值，这里source中包含了不同年份的值，years变量用于识别不同堆叠层
    x='fruits',  # 设置x坐标
    source=source,
    width=0.9,
    color=colors,
    legend=[value(x) for x in years],
    name=years)
# 绘制堆叠图
# 注意第一个参数需要放years

p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "horizontal"
# 设置其他参数

show(p)

In [28]:
# 3、堆叠图

from bokeh.palettes import GnBu3, OrRd3
# 导入颜色模块

fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
years = ["2015", "2016", "2017"]
exports = {
    'fruits': fruits,
    '2015': [2, 1, 4, 3, 2, 4],
    '2016': [5, 3, 4, 2, 4, 6],
    '2017': [3, 2, 4, 4, 5, 3]
}
imports = {
    'fruits': fruits,
    '2015': [-1, 0, -1, -3, -2, -1],
    '2016': [-2, -1, -3, -1, -2, -2],
    '2017': [-1, -2, -1, 0, -2, -2]
}

In [29]:
p = figure(y_range=fruits,
           plot_height=350,
           x_range=(-16, 16),
           title="Fruit import/export by year")
p.hbar_stack(years,
             y='fruits',
             height=0.9,
             color=GnBu3,
             source=ColumnDataSource(exports),
             legend=['%s exports' % x for x in years])
p.hbar_stack(years,
             y='fruits',
             height=0.9,
             color=OrRd3,
             source=ColumnDataSource(imports),
             legend=['%s imports' % x for x in years])

p.y_range.range_padding = 0.2
p.ygrid.grid_line_color = None
p.legend.location = 'top_left'
p.axis.minor_tick_line_color = None
p.outline_line_color = None

show(p)

In [30]:
# 4、直方图
# np.histogram + figure.quad()
# 不需要构建ColumnDataSource对象

df = pd.DataFrame({'value': np.random.randn(1000) * 100})
df.index.name = 'index'
print(df.head())
# 创建数据

            value
index            
0       52.025320
1       73.254116
2     -163.894636
3      -32.371624
4     -221.948255


In [31]:
hist, edges = np.histogram(df['value'], bins=20)
# 将数据解析成直方图统计格式
# 高阶函数np.histogram(a, bins=10, range=None, weights=None, density=None)
# a：数据
# bins：箱数
# range：最大最小值的范围，如果不设定则为(a.min(), a.max())
# weights：权重
# density：为True则返回“频率”，为False则返回“计数”
# 返回值1 - hist：每个箱子的统计值（top）
# 返回值2 - edges：每个箱子的位置坐标，这里n个bins将会有n+1个edges
print(hist)
print(edges)

[  2   4   6  15  30  35  75 103 126 130 112 106  91  71  46  24  17   4
   2   1]
[-331.326784   -298.25674383 -265.18670366 -232.11666349 -199.04662332
 -165.97658314 -132.90654297  -99.8365028   -66.76646263  -33.69642246
   -0.62638229   32.44365788   65.51369806   98.58373823  131.6537784
  164.72381857  197.79385874  230.86389891  263.93393908  297.00397926
  330.07401943]


In [32]:
p = figure(title='HIST', tools='save', background_fill_color='#E8DDCB')
p.quad(
    top=hist,
    bottom=0,
    left=edges[:-1],
    right=edges[1:],  # 分别代表每个柱子的四边值
    fill_color='#036564',
    line_color='#033649')
show(p)